<a href="https://colab.research.google.com/github/wallykop/IAD_SMMO_2019/blob/master/Kopylov_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## ИАД KAGGLE тексты
Решение для [соревнования](https://www.kaggle.com/c/iad-deep-learning-sentiment) на курсе СММО ИАД 2019

[Папка](https://drive.google.com/drive/folders/1kHOgfrSSR1LBxIABHZ8Z03nUI7Jo0ExW?usp=sharing) с файлами для загрузки:

In [0]:
from google.colab import drive
drive.mount('drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at drive


In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import string
import re

In [0]:
from keras.layers import Dense, Input, GRU, Embedding, Dropout, Bidirectional, BatchNormalization, LSTM, Activation, CuDNNGRU, CuDNNLSTM
from keras.layers import Bidirectional, GlobalMaxPool1D, Conv1D, GlobalMaxPooling1D, concatenate, SpatialDropout1D, Add, Flatten, BatchNormalization, GlobalAveragePooling1D
from keras.models import Model, Sequential
from keras.optimizers import Adam
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, LearningRateScheduler
from tqdm import tqdm_notebook

from keras.preprocessing import text, sequence
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences


from keras.models import Model, load_model

from keras import initializers, regularizers, constraints, optimizers, layers, callbacks
from keras.callbacks import Callback

# Сохраним модель
import cv2
import keras
from keras.models import save_model, load_model

def load_from_file(model_filename, last_epoch):
        model = load_model(model_filename)
        epoch = last_epoch
        return model

class ModelSaveCallback(keras.callbacks.Callback):

    def __init__(self, file_name):
        super(ModelSaveCallback, self).__init__()
        self.file_name = file_name


    def on_epoch_end(self, epoch, logs=None):
        epoch += 1
        model.save(self.file_name.format(epoch))
        print(model.save(self.file_name.format(epoch)))



Using TensorFlow backend.


In [0]:
x_train = pd.read_csv('/content/drive/My Drive/Sentiment_analysis_Kopylov/x_train.txt', sep = '\r\t', header = None, names = ['Review'])
x_test = pd.read_csv('/content/drive/My Drive/Sentiment_analysis_Kopylov/x_test.txt', sep = '\r\t', header = None, names = ['Review'])
y_train = pd.read_csv('/content/drive/My Drive/Sentiment_analysis_Kopylov/y_train.csv')
y_train = y_train.Probability.values

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


In [0]:
embed_size = 300 
max_features = 100000 
maxlen = 150

tokenizer = Tokenizer(num_words=max_features, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', lower=True,)
list_sentences_train = x_train.Review.values
list_sentences_test = x_test.Review.values
tokenizer.fit_on_texts(list(np.concatenate((list_sentences_train, list_sentences_test))))
list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)
x_train_pad = pad_sequences(list_tokenized_train, maxlen=maxlen)
x_test_pad = pad_sequences(list_tokenized_test, maxlen=maxlen)

In [0]:
from keras import backend as K
from keras.engine.topology import Layer
from keras import initializers, regularizers, constraints, optimizers, layers

class Attention(Layer):
    def __init__(self, step_dim,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        self.step_dim = step_dim
        self.features_dim = 0
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        self.features_dim = input_shape[-1]

        if self.bias:
            self.b = self.add_weight((input_shape[1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        else:
            self.b = None

        self.built = True

    def compute_mask(self, input, input_mask=None):
        return None

    def call(self, x, mask=None):
        features_dim = self.features_dim
        step_dim = self.step_dim

        eij = K.reshape(K.dot(K.reshape(x, (-1, features_dim)),
                        K.reshape(self.W, (features_dim, 1))), (-1, step_dim))

        if self.bias:
            eij += self.b

        eij = K.tanh(eij)

        a = K.exp(eij)

        if mask is not None:
            a *= K.cast(mask, K.floatx())

        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0],  self.features_dim

In [0]:
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import LearningRateScheduler

sequence_input = Input(shape=(maxlen,), dtype='int32')
embedding_layer = Embedding(max_features,
                            embed_size,
                            input_length=maxlen)
x = embedding_layer(sequence_input)

x = BatchNormalization()(x)
x = SpatialDropout1D(0.2)(x)
x = Bidirectional(CuDNNLSTM(128, return_sequences=True, kernel_initializer='lecun_uniform', bias_initializer='lecun_uniform'))(x)
x = Bidirectional(CuDNNLSTM(128, return_sequences=True, kernel_initializer='lecun_uniform', bias_initializer='lecun_uniform'))(x)
x = BatchNormalization()(x)

att = Attention(maxlen)(x)
att = BatchNormalization()(att)
att = Dropout(0.3)(att)
avg_pool1 = GlobalAveragePooling1D()(x)
max_pool1 = GlobalMaxPooling1D()(x)

x = concatenate([att,avg_pool1, max_pool1])

x = BatchNormalization()(x)
x = Dropout(0.3)(x)
x = Dense(512, kernel_initializer='lecun_uniform', bias_initializer='lecun_uniform')(x)

x = BatchNormalization()(x)
x = Dropout(0.2)(x)
x = Dense(128, kernel_initializer='lecun_uniform', bias_initializer='lecun_uniform')(x)

x = BatchNormalization()(x)
x = Dropout(0.1)(x)

preds = Dense(1, activation='sigmoid', kernel_initializer='lecun_uniform', bias_initializer='lecun_uniform')(x)

model = Model(sequence_input, preds)



def time_decay(epoch, initial_lrate):
    decay_rate = 0.05
    new_lrate = initial_lrate/(1+decay_rate*(epoch))
    return new_lrate

def schedule(ind):
    a = [0.0001, 0.0001, 0.00005, 0.00005, 0.00001, 0.00001, 0.000005, 0.000005, 0.000001, 0.000001]
    return a[ind]

lr = LearningRateScheduler(schedule)

lrate = LearningRateScheduler(time_decay,verbose=1)

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor = 0.5, patience = 2, min_lr = 0.0001)

early_stop = EarlyStopping(monitor='val_loss',
                           patience=5,
                           verbose=1,
                           min_delta=1e-4)

print(model.summary())
model.compile(loss = "binary_crossentropy", optimizer = Adam(lr = 0.001, decay = 0), metrics = ["accuracy"])






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 150)          0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 150, 300)     30000000    input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 150, 300)     1200        embedding_1[0][0]                
__________________________________________________________________________________________________
spatial_dropout1d_1 (SpatialDro (None, 150, 300)     0           batch_n

In [0]:
model_filename = 'weights_{0:02d}.hdf5'
history = model.fit(x_train_pad, 
                    y_train, 
                    batch_size = 512, 
                    epochs = 1, 
                    validation_split = 0.2, 
                    verbose = 1, callbacks = [ModelSaveCallback(model_filename), 
                                              lrate, 
                                              #lr,
                                              reduce_lr,
                                              early_stop])




Train on 2880000 samples, validate on 720000 samples
Epoch 1/1






Epoch 00001: LearningRateScheduler setting learning rate to 0.0010000000474974513.
2880000/2880000 [==============================] - 1021s 354us/step - loss: 0.1482 - acc: 0.9442 - val_loss: 0.1253 - val_acc: 0.9540
None


In [0]:
submission = model.predict(x_test_pad)
predictions = submission
answer = pd.DataFrame(predictions)
answer = pd.DataFrame({
    'Id': range(len(answer)),
    'Probability':answer[0]})
answer.Id = answer.Id + 1
answer.to_csv('predictions.csv', index = False)